In [1]:
import pandas as pd
import numpy as np

## Data

In [2]:
data = pd.read_csv('crsp_testset.csv')

In [3]:
data["RET"] = data["RET"].replace('C',0) # Corrupted return data set to 0
data["RET"] = data["RET"].astype('float64') # Conversion of RET: object -> float64 (string -> numeric)
data["ME"] = data["PRC"] * data["SHROUT"] / 1000
print("Min and max of ME: ({},{})".format(data["ME"].min(),(data["ME"].max())))

Min and max of ME: (0.13806000000000002,1304764.72281735)


In [4]:
def date_conversion(date):
    # Changing date format DDMMMYYYY -> YYYYMM i.e. 30JAN2015 -> 201501
    c = {'JAN' : '01', 'FEB' : '02', 'MAR' : '03', 'APR' : '04',
         'MAY' : '05', 'JUN' : '06', 'JUL' : '07', 'AUG' : '08',
         'SEP' : '09', 'OCT' : '10', 'NOV' : '11', 'DEC' : '12'}
    return date[5:] + c[date[2:5]]

v_date_conversion = np.vectorize(date_conversion)

In [5]:
print("Number of unique tickers: {}".format(data["TICKER"].nunique()))
data["date"] = v_date_conversion(data["date"])
print("Data range (date): {} to {}".format(data["date"].unique()[0], data["date"].unique()[-1]))
months = data["date"].unique()
print("Number of months: {}".format(months.size))
months

Number of unique tickers: 5058
Data range (date): 201501 to 201912
Number of months: 60


array(['201501', '201502', '201503', '201504', '201505', '201506',
       '201507', '201508', '201509', '201510', '201511', '201512',
       '201601', '201602', '201603', '201604', '201605', '201606',
       '201607', '201608', '201609', '201610', '201611', '201612',
       '201701', '201702', '201703', '201704', '201705', '201706',
       '201707', '201708', '201709', '201710', '201711', '201712',
       '201801', '201802', '201803', '201804', '201805', '201806',
       '201807', '201808', '201809', '201810', '201811', '201812',
       '201901', '201902', '201903', '201904', '201905', '201906',
       '201907', '201908', '201909', '201910', '201911', '201912'],
      dtype=object)

In [6]:
def print_ret_table(df,first=3,last=3): return df.head(first).append(df.tail(last)) 
# helper for printing return tables for debugging

## Multiprocessing

In [7]:
import multiprocessing as mp
from multiprocessing.pool import ThreadPool

In [8]:
def threadify(f, arr):
    # Maps array with function f parallelized
    pool = ThreadPool(mp.cpu_count())
    result = pool.map(f,arr)
    pool.terminate()
    pool.join()
    return result

## Wrangling

In [9]:
pd.options.mode.chained_assignment = None

def lag_return_permno(data,n,permno):
    # Return df with added lagged returns (up to n lags) of a stock (PERMNO)
    # Loses n first observations 
    permno_data = data.loc[data['PERMNO'] == permno]
    for lag in range(1,n+1):
        col = 'LRET_{}'.format(lag)
        permno_data[col] = permno_data['RET'].shift(lag)
    return permno_data.iloc[3:]

def lag_returns(data,n):
    # Adds lagged returns (up to n lags) of each stock to the dataset
    # Loses n first observations for each stock
    permnos = data['PERMNO'].unique()
    dfs = threadify(lambda permno: lag_return_permno(data,n,permno),permnos)
    return pd.concat(dfs, ignore_index=True)

In [10]:
lagged_data = lag_returns(data,3)

In [11]:
def quantiles_for_date(data,wrt,date,r):
    # return list of breakpoint values (quantiles) on a date wrt to a given variable
    # r = quantile separation, e.g. n = 20 -> [q(0.2),q(0.4),q(0.6),q(0.8),q(1)]
    data = data.loc[data['date'] == date]
    qs = [q/100 for q in range(r,100+r,r)]
    breakpoints = [np.quantile(data[wrt].values,q) for q in qs]
    return {**{'date':date}, **dict(zip(range(1,r+1),breakpoints))}

def quantile_table(data,wrt,r):
    dates = data['date'].unique()
    qs_dates = threadify(lambda date: quantiles_for_date(data,wrt,date,r),dates)
    return pd.DataFrame(qs_dates).set_index('date')

In [12]:
quantile_table(lagged_data,'LRET_1',10).head()

,1,2,3,4,5,6,7,8,9,10
date,,,,,,,,,,
201504,-0.104745,-0.055241,-0.028374,-0.010547,0.003913,0.020808,0.038520,0.064205,0.112004,2.025783
201505,-0.111171,-0.069818,-0.046071,-0.028331,-0.012776,0.002763,0.023269,0.051835,0.111181,9.564357
201506,-0.102128,-0.051048,-0.023609,-0.005460,0.008096,0.023127,0.041088,0.069362,0.128765,1.793594
201507,-0.100592,-0.062148,-0.039800,-0.020046,-0.002456,0.016041,0.036203,0.062969,0.103392,0.930233
201508,-0.160441,-0.096924,-0.063009,-0.035843,-0.014799,0.003454,0.024891,0.053396,0.097946,1.651020
